In [1]:
# IMPORTS

from bs4 import BeautifulSoup
import bs4
import requests
from selenium import webdriver

import pandas as pd
import numpy as np

In [2]:
# USING SPACY FOR WEB SCRAPPING

driver = webdriver.Chrome("chromedriver.exe")

In [3]:
# LIST OF UNIQUE CATEGORIES THAT WAS NEEDED FOR THE DATASET.
# ADD OR SUBTRACT LABELS AS NEEDED

job_list = ['HR', 'designer', 'Information-Technology',
       'Teacher', 'Advocate', 'Business-Development',
       'Healthcare', 'Fitness', 'Agriculture', 'BPO', 'Sales', 'Consultant',
       'Digital-Media', 'Automobile', 'Chef', 'Finance',
       'Apparel', 'Engineering', 'Accountant', 'Construction',
       'Public-Relations', 'Banking', 'Arts', 'Aviation']

In [4]:
# CREATING A DATAFRAME TO STORE THE LINKS OF EACH INDIVIDUAL RESUME EXAMPLE

resume_links = pd.DataFrame()
category = []
link = []

In [5]:
# LOOP TO SEARCH FOR RESUME EXAMPLES FROM THE WEBPAGE AS PER THE LABELS DEFINED ABOVE. ONLY (10*12) 120 RESUMES WILL BE COLLECTED.

# REMOVE  "&bg=85&eg=100&comp=&mod="  TO EXPAND THE SEARCH 
# (bg=85 & eg=100) FILTERS THE RESUME BASED ON SCORE OF BETWEEEN 85 AND 100, INCREASE RANGE TO EXPAND SEARCH

for job in job_list:
    JOB = job.lower()
    for i in range(1,13):   # INCREASE THE RANGE TO GET MORE RESUME DATA
        PAGE = str(i)
        URL = "https://www.livecareer.com/resume-search/search?jt=" + JOB + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
        driver.get(URL)
        aTagsInLi = driver.find_elements_by_css_selector('li a')
        for a in aTagsInLi:
            if a.get_attribute('rel') == "ugc":
                category.append(JOB)
                link.append(a.get_attribute('href'))
    

In [6]:
# STORES THE CATEGORY AND LINK TO THE RESUME WEBPAGE

resume_links["Category"] = category
resume_links["link"] = link

In [7]:
# HASHES THE LINK AND CREATES AN UNIQUE ID FOR THE ROWS

import hashlib
def id(x):
    return int(hashlib.md5(x.encode('utf-8')).hexdigest(), 16)

resume_links["id"] = resume_links["link"].apply(id)

In [8]:
# NUMBER OF RESUME COLLECTED PER DEFINED LABELS

df.Category.value_counts()

In [10]:
df["Resume"] = ""
df["Raw_html"] = ""

In [11]:
# LOOP TO COLLECT THE RESUME DATA PRESENT IN THE LINKS AS COLECTED ABOVE
# STORED IN HTML AS WELL AS STRING FORMAT

for i in range(df.shape[0]):
    url = df.link[i]
    driver.get(url)
#     time.sleep(0.5)                  #ADDED DELAY, CAN BE REMOVED
    x = driver.page_source
    x = x.replace(">","> ")
    soup = bs4.BeautifulSoup(x, 'html.parser')
    div = soup.find("div", {"id": "document"})
    df.Raw_html[i] = div
    try:
        df.Resume[i] = div.text
    except:
#         ADD EXCEPTION IF REQUIRED
        pass



In [20]:
df.head()

,Category,link,id,Resume,Raw_html
0,hr,https://www.livecareer.com/resume-search/r/hr-...,128216776680979599862713391059783413195,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"[ , [ , [ , <div class=""name"" itemprop=""name"">..."
1,hr,https://www.livecareer.com/resume-search/r/hr-...,201122761231557277513252090847674597006,"HR SPECIALIST, US HR OPERATIONS ...","[ , [ , [ , <div class=""name thinbottomborder""..."
2,hr,https://www.livecareer.com/resume-search/r/hr-...,317423129318109781526148386940790993392,HR DIRECTOR Summary Over 2...,"[ , [ , [ , <div class=""name"" itemprop=""name"">..."
3,hr,https://www.livecareer.com/resume-search/r/hr-...,200997734602806164328411756164577760181,HR SPECIALIST Summary Dedica...,"[ , [ , [ , <div class=""name"" itemprop=""name"">..."
4,hr,https://www.livecareer.com/resume-search/r/hr-...,162057789781141134402378787192982207371,HR MANAGER Skill Highlights ...,"[ , [ , [ , <div class=""name"" itemprop=""name"">..."


In [21]:
# STORES THE DATAFRAME

df.to_csv("Resume.csv",index=False)